In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241796
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-241796


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
 
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--max_iter': choice(100, 500, 1000),
        '--C': uniform(1e-6, 1e6)
    }
) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4) ### YOUR CODE HERE ###

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [11]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_c506b459-0d31-4e26-bc66-8a236e788440
Web View: https://ml.azure.com/runs/HD_c506b459-0d31-4e26-bc66-8a236e788440?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-241796/workspaces/quick-starts-ws-241796&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-25T15:51:07.652741][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-09-25T15:51:08.3336523Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_0' 
[2023-09-25T15:51:08.4636924Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_1' 
[2023-09-25T15:51:08.6885358Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_3' 
[2023-09-25T15:51:08.5844793Z][SCHEDULER][INFO]Scheduling job, id='HD_c506b459-0d31-4e26-bc66-8a236e788440_2' 
[2023-09-25T15:51:08.638513][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

In [38]:
assert(hyperdrive_run.get_status() == "Completed")

In [17]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [20]:
best_run.get_metrics()

{'Regularization Strength:': 776525.935251541,
 'Max iterations:': 500,
 'Accuracy': 0.9059237679048313}

In [19]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-09-25-16',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [22]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_link)

In [23]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [29]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    X=x,
    y=y,
    n_cross_validations=5)

In [30]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e21457fd-b9e3-4466-8878-a24b5fc60f51,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/09/25 16:22:24 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [31]:
automl_run.wait_for_completion(show_output = True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e21457fd-b9e3-4466-8878-a24b5fc60f51,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_e21457fd-b9e3-4466-8878-a24b5fc60f51',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-09-25T16:21:38.069329Z',
 'endTimeUtc': '2023-09-25T16:53:41.759505Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-241796","workspace_name":"quick-starts-ws-241796","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable

In [32]:
assert(automl_run.get_status() == "Completed")

In [33]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_automl_model = automl_run.get_output()

In [34]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e21457fd-b9e3-4466-8878-a24b5fc60f51_29,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
fitted_automl_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('25', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('

In [40]:
joblib.dump(fitted_automl_model, 'automl.joblib')

['automl.joblib']

In [37]:
aml_compute.delete()